In [ ]:
from netCDF4 import Dataset

In [ ]:
file = '/home/heqin/Documents/dataAssimilationNumbaBigFile/wrfout_d02_2015-12-26_00_00_00'

In [ ]:
help(Dataset)

In [ ]:
readWRF = Dataset(file,'r')

In [ ]:
P_wrf = readWRFreadNC.variables['P'][0] #由于只有这一个时间维，这儿就肯定先放上0

In [ ]:
print(P_wrf[0]) #这儿的0是层数

下面将上面的代码整理出来
# 计算wrf的总柱浓度

In [ ]:
from netCDF4 import Dataset
file = '/home/heqin/Documents/dataAssimilationNumbaBigFile/wrfout_d02_2015-12-26_00_00_00'
with Dataset(file,'r') as readWRF:
    P_wrf = readWRF.variables['P'][0]
    PB_wrf = readWRF.variables['PB'][0]
    P = P_wrf + PB_wrf
    
    T_wrf = readWRF.variables['T'][0]
    T_potential = T_wrf + 300
    T = T_potential*(P/100000)**0.2854
    
    #计算每层单位转换系数
    r_to_n = 7.244E+10*P/T #体积混合比r（ppm）转到数浓度n（molec./cm3）
    
    r_wrfNO2 = readWRF.variables['no2'][0] #单独列出H时其实用不到它，n是数浓度
    
    #计算每层的高度和每层柱浓度
    height = []
    r_to_column = []
    for l in range(30):
        Up_wrf = readWRF.variables['PH'][0][l+1] + readWRF.variables['PHB'][0][l+1]
        Bottom_wrf = readWRF.variables['PH'][0][l] + readWRF.variables['PHB'][0][l]
        height.append((Up_wrf-Bottom_wrf)/9.81)
        r_to_column.append(r_to_n[l]*height[l])
        
    #计算总的柱浓度
    column_wrfNO2 = 0
    for l in range(30):
        temp = r_wrfNO2[l] * r_to_column[l]
        print(temp[38][68])
        column_wrfNO2 = temp + column_wrfNO2
        print(column_wrfNO2[38][68])

# 确定位置 (排序算法有点错)

In [13]:
# -*- coding:utf-8 -*-
import os
import glob
import numba
from numba import jit
import h5py
import numpy as np
from math import radians, cos, sin, asin, sqrt 
from netCDF4 import Dataset

@jit
def haversine(lon1, lat1, lon2, lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）  
    """ 
    Calculate the great circle distance between two points  
    on the earth (specified in decimal degrees) 
    """  
    # 将十进制度数转化为弧度  
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])  
  
    # haversine公式  
    dlon = lon2 - lon1   
    dlat = lat2 - lat1   
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2  
    c = 2 * asin(sqrt(a))   
    r = 6371 # 地球平均半径，单位为公里  
    return c * r * 1000 # 最终单位为米
    
fileWRF = '/home/heqin/Documents/dataAssimilationNumbaBigFile/wrfout_d02_2015-12-26_00_00_00'
with Dataset(fileWRF,'r') as readWRF:  
    lat_wrf = readWRF.variables['XLAT'][0]
    lon_wrf = readWRF.variables['XLONG'][0]
    
fileSATE = '/home/heqin/Documents/dataAssimilationNumbaBigFile/S5P_OFFL_L2__NO2____20180716T044109_20180716T062239_03913_01_010100_20180722T060909.nc'

with h5py.File(fileSATE,'r') as readSATE:
    lat_sate = readSATE['PRODUCT']['latitude'][0]
    lon_sate = readSATE['PRODUCT']['longitude'][0]
    column_sateNO2 = readSATE['PRODUCT']['nitrogendioxide_tropospheric_column'][0]
    qa = readSATE['PRODUCT']['qa_value'][0]
    for ii in range(222,333): # 450
        for jj in range(1111,2222): # 2905需要设置成变量        
            if (qa[jj][ii]) == 100:
                dis = np.zeros((84,96))
                temp = np.zeros(84*96)
                for i in range(96):
                    for j in range(84):
                        # 对于第x个卫星像元，所有的WRF网格与它的距离
                        dis[j][i] = haversine(lon_sate[jj][ii], lat_sate[jj][ii], lon_wrf[j][i], lat_wrf[j][i])
                        temp[j*84+i] = dis[j][i]
                        
                sort_temp = np.argsort(temp).tolist()
                a = sort_temp.index(0)
                print(ii,jj,'++++++++++++++++++++++','SATEend')
                if temp[a] < 63636 :
                    b = sort_temp.index(1)
                    c = sort_temp.index(2)
                    d = sort_temp.index(3)
                
                    print(a,b,c,d)
                    print(a//84,b//84,c//84,d//84)
                    print(a%84,b%84,c%84,d%84)
                    


    
    

    
    
    '''
    
    
dis =haversine(lon_sate, lat_sate, lon_wrf, lat_wrf)
        
        
    #单纯的观测算子需要将r_to_column展开
    for i in range(84):
        for j in range(96):
            column_sateNO2 = 
            position_i = 
            position_j = 
            
            '''

222 1111 ++++++++++++++++++++++ SATEend
222 1112 ++++++++++++++++++++++ SATEend
222 1113 ++++++++++++++++++++++ SATEend
222 1114 ++++++++++++++++++++++ SATEend
222 1115 ++++++++++++++++++++++ SATEend
222 1116 ++++++++++++++++++++++ SATEend
222 1117 ++++++++++++++++++++++ SATEend
222 1118 ++++++++++++++++++++++ SATEend
222 1119 ++++++++++++++++++++++ SATEend
222 1120 ++++++++++++++++++++++ SATEend
222 1121 ++++++++++++++++++++++ SATEend
222 1122 ++++++++++++++++++++++ SATEend
222 1123 ++++++++++++++++++++++ SATEend
222 1124 ++++++++++++++++++++++ SATEend
222 1125 ++++++++++++++++++++++ SATEend
222 1126 ++++++++++++++++++++++ SATEend
222 1127 ++++++++++++++++++++++ SATEend
222 1128 ++++++++++++++++++++++ SATEend
222 1129 ++++++++++++++++++++++ SATEend
222 1130 ++++++++++++++++++++++ SATEend
222 1131 ++++++++++++++++++++++ SATEend
222 1132 ++++++++++++++++++++++ SATEend
222 1133 ++++++++++++++++++++++ SATEend
222 1134 ++++++++++++++++++++++ SATEend
222 1135 ++++++++++++++++++++++ SATEend


222 1883 ++++++++++++++++++++++ SATEend
222 1884 ++++++++++++++++++++++ SATEend
222 1885 ++++++++++++++++++++++ SATEend
222 1889 ++++++++++++++++++++++ SATEend
222 1890 ++++++++++++++++++++++ SATEend
222 1891 ++++++++++++++++++++++ SATEend
222 1892 ++++++++++++++++++++++ SATEend
222 1897 ++++++++++++++++++++++ SATEend
222 1898 ++++++++++++++++++++++ SATEend
222 1899 ++++++++++++++++++++++ SATEend
222 1904 ++++++++++++++++++++++ SATEend
222 1905 ++++++++++++++++++++++ SATEend
222 1906 ++++++++++++++++++++++ SATEend
222 1907 ++++++++++++++++++++++ SATEend
222 1909 ++++++++++++++++++++++ SATEend
222 1910 ++++++++++++++++++++++ SATEend
222 1911 ++++++++++++++++++++++ SATEend
222 1912 ++++++++++++++++++++++ SATEend
222 1913 ++++++++++++++++++++++ SATEend
222 1914 ++++++++++++++++++++++ SATEend
222 1915 ++++++++++++++++++++++ SATEend
222 1916 ++++++++++++++++++++++ SATEend
222 1917 ++++++++++++++++++++++ SATEend
222 1919 ++++++++++++++++++++++ SATEend
222 1920 ++++++++++++++++++++++ SATEend


223 1623 ++++++++++++++++++++++ SATEend
223 1624 ++++++++++++++++++++++ SATEend
223 1625 ++++++++++++++++++++++ SATEend
223 1626 ++++++++++++++++++++++ SATEend
223 1627 ++++++++++++++++++++++ SATEend
223 1628 ++++++++++++++++++++++ SATEend
223 1632 ++++++++++++++++++++++ SATEend
223 1633 ++++++++++++++++++++++ SATEend
223 1634 ++++++++++++++++++++++ SATEend
223 1639 ++++++++++++++++++++++ SATEend
223 1640 ++++++++++++++++++++++ SATEend
223 1668 ++++++++++++++++++++++ SATEend
223 1672 ++++++++++++++++++++++ SATEend
223 1673 ++++++++++++++++++++++ SATEend
223 1674 ++++++++++++++++++++++ SATEend
223 1675 ++++++++++++++++++++++ SATEend
223 1676 ++++++++++++++++++++++ SATEend
223 1677 ++++++++++++++++++++++ SATEend
223 1678 ++++++++++++++++++++++ SATEend
223 1679 ++++++++++++++++++++++ SATEend
223 1680 ++++++++++++++++++++++ SATEend
223 1681 ++++++++++++++++++++++ SATEend
223 1682 ++++++++++++++++++++++ SATEend
223 1683 ++++++++++++++++++++++ SATEend
223 1684 ++++++++++++++++++++++ SATEend


223 2142 ++++++++++++++++++++++ SATEend
224 1111 ++++++++++++++++++++++ SATEend
224 1112 ++++++++++++++++++++++ SATEend
224 1113 ++++++++++++++++++++++ SATEend
224 1114 ++++++++++++++++++++++ SATEend
224 1115 ++++++++++++++++++++++ SATEend
224 1116 ++++++++++++++++++++++ SATEend
224 1117 ++++++++++++++++++++++ SATEend
224 1118 ++++++++++++++++++++++ SATEend
224 1119 ++++++++++++++++++++++ SATEend
224 1120 ++++++++++++++++++++++ SATEend
224 1121 ++++++++++++++++++++++ SATEend
224 1122 ++++++++++++++++++++++ SATEend
224 1123 ++++++++++++++++++++++ SATEend
224 1124 ++++++++++++++++++++++ SATEend
224 1125 ++++++++++++++++++++++ SATEend
224 1126 ++++++++++++++++++++++ SATEend
224 1127 ++++++++++++++++++++++ SATEend
224 1128 ++++++++++++++++++++++ SATEend
224 1129 ++++++++++++++++++++++ SATEend
224 1130 ++++++++++++++++++++++ SATEend
224 1131 ++++++++++++++++++++++ SATEend
224 1132 ++++++++++++++++++++++ SATEend
224 1133 ++++++++++++++++++++++ SATEend
224 1134 ++++++++++++++++++++++ SATEend


224 1899 ++++++++++++++++++++++ SATEend
224 1900 ++++++++++++++++++++++ SATEend
224 1905 ++++++++++++++++++++++ SATEend
224 1906 ++++++++++++++++++++++ SATEend
224 1907 ++++++++++++++++++++++ SATEend
224 1908 ++++++++++++++++++++++ SATEend
224 1909 ++++++++++++++++++++++ SATEend
224 1910 ++++++++++++++++++++++ SATEend
224 1911 ++++++++++++++++++++++ SATEend
224 1912 ++++++++++++++++++++++ SATEend
224 1913 ++++++++++++++++++++++ SATEend
224 1914 ++++++++++++++++++++++ SATEend
224 1915 ++++++++++++++++++++++ SATEend
224 1917 ++++++++++++++++++++++ SATEend
224 1918 ++++++++++++++++++++++ SATEend
224 1919 ++++++++++++++++++++++ SATEend
224 1920 ++++++++++++++++++++++ SATEend
224 1921 ++++++++++++++++++++++ SATEend
224 1922 ++++++++++++++++++++++ SATEend
224 1923 ++++++++++++++++++++++ SATEend
224 1924 ++++++++++++++++++++++ SATEend
224 1925 ++++++++++++++++++++++ SATEend
224 1926 ++++++++++++++++++++++ SATEend
224 1927 ++++++++++++++++++++++ SATEend
224 1928 ++++++++++++++++++++++ SATEend


KeyboardInterrupt: 

# 确定位置 (单像元的位置)

In [32]:
# -*- coding:utf-8 -*-
import os
import glob
import numba
from numba import jit
import h5py
import numpy as np
from math import radians, cos, sin, asin, sqrt 
from netCDF4 import Dataset

@jit
def haversine(lon1, lat1, lon2, lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）  
    """ 
    Calculate the great circle distance between two points  
    on the earth (specified in decimal degrees) 
    """  
    # 将十进制度数转化为弧度  
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])  
  
    # haversine公式  
    dlon = lon2 - lon1   
    dlat = lat2 - lat1   
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2  
    c = 2 * asin(sqrt(a))   
    r = 6371 # 地球平均半径，单位为公里  
    return c * r * 1000 # 最终单位为米
    
fileWRF = '/home/heqin/Documents/dataAssimilationNumbaBigFile/wrfout_d02_2015-12-26_00_00_00'
with Dataset(fileWRF,'r') as readWRF:  
    lat_wrf = readWRF.variables['XLAT'][0]
    lon_wrf = readWRF.variables['XLONG'][0]
    
fileSATE = '/home/heqin/Documents/dataAssimilationNumbaBigFile/S5P_OFFL_L2__NO2____20180716T044109_20180716T062239_03913_01_010100_20180722T060909.nc'

with h5py.File(fileSATE,'r') as readSATE:
    lat_sate = readSATE['PRODUCT']['latitude'][0]
    lon_sate = readSATE['PRODUCT']['longitude'][0]
    column_sateNO2 = readSATE['PRODUCT']['nitrogendioxide_tropospheric_column'][0]
    qa = readSATE['PRODUCT']['qa_value'][0]
    with open('SATE.txt','w') as writeSATE:
        for ii in range(450): # 450
            for jj in range(2905): # 2905需要设置成变量        
                if (qa[jj][ii]) == 100:
                    if 30<(lat_sate[jj][ii])<45:
                        if 105<(lon_sate[jj][ii])<125:
                            if column_sateNO2[jj][ii] >0:
                                print(jj,ii,lat_sate[jj][ii],lon_sate[jj][ii],column_sateNO2[jj][ii],file=writeSATE)
                '''
                dis = np.zeros((84,96))
                temp = np.zeros(84*96)
                for i in range(96):
                    for j in range(84):
                        # 对于第x个卫星像元，所有的WRF网格与它的距离
                        dis[j][i] = haversine(lon_sate[jj][ii], lat_sate[jj][ii], lon_wrf[j][i], lat_wrf[j][i])
                        temp[j*84+i] = dis[j][i]
                        
                sort_temp = np.argsort(temp).tolist()
                a = sort_temp.index(0)
                #print(ii,jj,'++++++++++++++++++++++','SATEend')
                if temp[a] < 63636 :
                    print(ii,jj,a,a//84,a%84,temp[a],lon_sate[a//84][a%84], lat_sate[a//84][a%84], lon_wrf[j][i], lat_wrf[j][i])
                '''

In [19]:
print(haversine(lon_sate[1929][222], lat_sate[1929][222], lon_wrf[81][14], lat_wrf[81][14]))
print(haversine(lon_sate[1929][222], lat_sate[1929][222], lon_wrf[80][14], lat_wrf[80][14]))

print(haversine(lon_sate[1929][222], lat_sate[1929][222], lon_wrf[80][80], lat_wrf[80][80]))
print(haversine(lon_sate[1929][222], lat_sate[1929][222], lon_wrf[80][79], lat_wrf[80][79]))
print(haversine(lon_sate[1929][222], lat_sate[1929][222], lon_wrf[80][63], lat_wrf[80][63]))
print(haversine(lon_sate[1929][222], lat_sate[1929][222], lon_wrf[80][64], lat_wrf[80][64]))
print(haversine(lon_sate[1929][222], lat_sate[1929][222], lon_wrf[80][45], lat_wrf[80][45]))


40956.35359215609
33809.90303960308
992612.8603461119
977084.0635150541
728643.2288050258
744169.9280142837
449126.3772045263


# 除了确定位置以外的观测算子

In [ ]:
from netCDF4 import Dataset
file = '/home/heqin/Documents/dataAssimilationNumbaBigFile/wrfout_d02_2015-12-26_00_00_00'
with Dataset(file,'r') as readWRF:
    P_wrf = readWRF.variables['P'][0]
    PB_wrf = readWRF.variables['PB'][0]
    P = P_wrf + PB_wrf
    
    T_wrf = readWRF.variables['T'][0]
    T_potential = T_wrf + 300
    T = T_potential*(P/100000)**0.2854
    
    #计算每层单位转换系数
    r_to_n = 7.244E+10*P/T #体积混合比r（ppm）转到数浓度n（molec./cm3）
    
    r_wrfNO2 = readWRF.variables['no2'][0] #单独列出H时其实用不到它，n是数浓度
    
    #计算每层的高度和每层柱浓度
    height = []
    r_to_column = []
    for l in range(30):
        Up_wrf = readWRF.variables['PH'][0][l+1] + readWRF.variables['PHB'][0][l+1]
        Bottom_wrf = readWRF.variables['PH'][0][l] + readWRF.variables['PHB'][0][l]
        height.append((Up_wrf-Bottom_wrf)/9.81)
        r_to_column.append(r_to_n[l]*height[l])
         
    #单纯的观测算子需要将r_to_column展开
    for i in range(84):
        for j in range(96):
            H_r_to_column[how many lines？][i*84+j] = r_to_column[j][i] #有多少个卫星观测点就有多少行

In [ ]:
test = np.ones((40000,40000))

In [1]:
from netCDF4 import Dataset
import numpy as np
zDA = 30
iDA = 84
jDA = 96
# = np.zeros((iDA,jDA,zDA))

file = '/home/heqin/Documents/dataAssimilationNumbaBigFile/wrfout_d02_2015-12-26_00_00_00'

with Dataset(file,'r') as readWRF:
    P_wrf = readWRF.variables['P'][0]
    PB_wrf = readWRF.variables['PB'][0]
    P = P_wrf + PB_wrf
    
    T_wrf = readWRF.variables['T'][0]
    T_potential = T_wrf + 300
    T = T_potential*(P/100000)**0.2854
    
    #计算每层单位转换系数
    r_to_n = 7.244E+10*P/T #体积混合比r（ppm）转到数浓度n（molec./cm3）
    
    r_wrfNO2 = readWRF.variables['no2'][0] #单独列出H时其实用不到它，n是数浓度
    
    #计算每层的高度和每层柱浓度
    height = []
    r_to_column = []
    for z in range(zDA):
        Up_wrf = readWRF.variables['PH'][0][z+1] + readWRF.variables['PHB'][0][z+1]
        Bottom_wrf = readWRF.variables['PH'][0][z] + readWRF.variables['PHB'][0][z]
        height.append((Up_wrf-Bottom_wrf)/9.81)
        r_to_column.append(r_to_n[z]*height[z])
        
    '''计算总的柱浓度
    column_wrfNO2 = 0
    for z in range(zDA):
        temp = r_wrfNO2[z] * r_to_column[z]
        #print(temp[38][68])
        column_wrfNO2 = temp + column_wrfNO2
        #print(column_wrfNO2[38][68])
    '''